In [3]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from pokemon_card_generator.data import card_data

card_df = card_data.get_card_data()

/home/phil/code/pokemon_card_generator/pokemon_card_generator


IndexError: list index out of range

In [5]:
card_df.columns

NameError: name 'card_df' is not defined

In [6]:
# list all the files in data
from os import walk, path

two_dirs_up = path.dirname(path.dirname(path.abspath(__file__)))
pickles_path = path.join(two_dirs_up, "raw_data/pickles")
    
filenames = next(walk(pickles_path), (None, None, []))[2]  # [] if no file

NameError: name '__file__' is not defined